# 1. import modules

In [4]:
import torch
import numpy as np

preprocess

In [5]:
# Load log records and crash sequences
log_records = np.load('log_records.npy')
print(log_records.shape)
print(len(log_records))

(242, 384)
242


In [18]:

crash_sequences = np.array([[0,0,1,1,1,0,0,0,0] for x in range(0, log_records.shape[0])])

# Split data into training and testing sets
train_ratio = 0.8
split_index = int(len(log_records) * train_ratio)

train_records = log_records[:split_index]
train_sequences = crash_sequences[:split_index]

test_records = log_records[split_index:]
test_sequences = crash_sequences[split_index:]


model def

In [19]:
class LogCrashModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LogCrashModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the dimensions for your model
input_dim = 384
hidden_dim = 128
output_dim = 9  # Assuming you have two classes (crash or no crash)

# Create an instance of the model
model = LogCrashModel(input_dim, hidden_dim, output_dim)


Define the loss function and optimizer

In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train


In [28]:
num_epochs = 10
batch_size = log_records.shape[0]//num_epochs

for epoch in range(num_epochs):
    permutation = np.random.permutation(len(train_records)).astype(int)
    train_records = train_records[permutation]
    train_sequences = train_sequences[permutation]

    for i in range(0, len(train_records), batch_size):
        batch_records = torch.tensor(train_records[i:i+batch_size]).float()
        batch_sequences = torch.tensor(train_sequences[i:i+batch_size]).float()

        # Forward pass
        outputs = model(batch_records)
        loss = criterion(outputs, batch_sequences)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(
                f'Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{len(train_records)}, Loss: {loss.item()}')
        


Epoch: 1/10, Step: 1/193, Loss: 3.2961719036102295
Epoch: 1/10, Step: 25/193, Loss: 3.297083616256714
Epoch: 1/10, Step: 49/193, Loss: 3.29618763923645
Epoch: 1/10, Step: 73/193, Loss: 3.2961666584014893
Epoch: 1/10, Step: 97/193, Loss: 3.2970762252807617
Epoch: 1/10, Step: 121/193, Loss: 3.2970612049102783
Epoch: 1/10, Step: 145/193, Loss: 3.296217679977417
Epoch: 1/10, Step: 169/193, Loss: 3.296178102493286
Epoch: 1/10, Step: 193/193, Loss: 3.296066999435425
Epoch: 2/10, Step: 1/193, Loss: 3.2961838245391846
Epoch: 2/10, Step: 25/193, Loss: 3.2962000370025635
Epoch: 2/10, Step: 49/193, Loss: 3.296144485473633
Epoch: 2/10, Step: 73/193, Loss: 3.296151876449585
Epoch: 2/10, Step: 97/193, Loss: 3.2961950302124023
Epoch: 2/10, Step: 121/193, Loss: 3.2970142364501953
Epoch: 2/10, Step: 145/193, Loss: 3.296982765197754
Epoch: 2/10, Step: 169/193, Loss: 3.2970120906829834
Epoch: 2/10, Step: 193/193, Loss: 3.2962660789489746
Epoch: 3/10, Step: 1/193, Loss: 3.2969751358032227
Epoch: 3/10, Ste

evaluate


In [39]:
test_records = torch.tensor(test_records).float()
test_sequences = torch.tensor(test_sequences).float()

with torch.no_grad():
    outputs = model(test_records)
    # crossentrophy loss
    loss = criterion(outputs, test_sequences)
    print(loss)


tensor(3.2964)


C:\Users\thinkbook\AppData\Local\Temp\ipykernel_63352\624818299.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_records = torch.tensor(test_records).float()
C:\Users\thinkbook\AppData\Local\Temp\ipykernel_63352\624818299.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_sequences = torch.tensor(test_sequences).float()
